In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import sdmetrics

import sdv as sdv
from sdv.datasets.local import load_csvs
from sdv.metadata import Metadata, SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
from sdv.evaluation.single_table import run_diagnostic
from sdv.evaluation.single_table import evaluate_quality
from sdv.evaluation.single_table import get_column_plot
from sdmetrics.single_column import KSComplement
from sdmetrics.single_column import TVComplement
from sdmetrics.visualization import get_column_plot
from sdmetrics.single_column import MissingValueSimilarity
from sdmetrics.single_table import NewRowSynthesis
from sdv.evaluation.single_table import get_column_pair_plot









In [12]:
file_path = './rhc.csv'
data = pd.read_csv(file_path)
data.head(10)


,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,1,COPD,NaN,Yes,11142,11151.0,NaN,11382,No,0,...,No,No,No,No,No,0.0,NaN,white,Under $11k,5
1,2,MOSF w/Sepsis,NaN,No,11799,11844.0,11844.0,11844,Yes,1,...,No,No,Yes,No,No,NaN,1437.0,white,Under $11k,7
2,3,MOSF w/Malignancy,MOSF w/Sepsis,Yes,12083,12143.0,NaN,12400,No,0,...,No,No,No,No,No,NaN,599.0,white,$25-$50k,9
3,4,ARF,NaN,No,11146,11183.0,11183.0,11182,Yes,0,...,No,No,No,No,No,NaN,NaN,white,$11-$25k,10
4,5,MOSF w/Sepsis,NaN,No,12035,12037.0,12037.0,12036,Yes,0,...,No,No,No,No,No,NaN,64.0,white,Under $11k,11
5,6,COPD,NaN,No,12389,12396.0,NaN,12590,No,0,...,No,No,No,No,No,0.0,242.0,white,Under $11k,12
6,7,MOSF w/Malignancy,NaN,Metastatic,12381,12423.0,NaN,12616,No,0,...,No,No,No,No,No,NaN,4465.0,white,$25-$50k,13
7,8,ARF,Coma,No,11453,11487.0,11491.0,11490,Yes,0,...,No,No,Yes,No,No,NaN,NaN,white,$25-$50k,14
8,9,MOSF w/Malignancy,NaN,Yes,12426,12437.0,NaN,12560,No,0,...,No,Yes,No,No,No,NaN,5525.0,white,Under $11k,16
9,10,ARF,NaN,Yes,11381,11400.0,NaN,11590,No,0,...,No,No,No,No,No,NaN,NaN,white,Under $11k,17


In [40]:
print(data.dtypes)

Unnamed: 0      int64
cat1           object
cat2           object
ca             object
sadmdte         int64
               ...   
adld3p        float64
urin1         float64
race           object
income         object
ptid            int64
Length: 63, dtype: object


In [13]:
data.keys()

Index(['Unnamed: 0', 'cat1', 'cat2', 'ca', 'sadmdte', 'dschdte', 'dthdte',
       'lstctdte', 'death', 'cardiohx', 'chfhx', 'dementhx', 'psychhx',
       'chrpulhx', 'renalhx', 'liverhx', 'gibledhx', 'malighx', 'immunhx',
       'transhx', 'amihx', 'age', 'sex', 'edu', 'surv2md1', 'das2d3pc',
       't3d30', 'dth30', 'aps1', 'scoma1', 'meanbp1', 'wblc1', 'hrt1', 'resp1',
       'temp1', 'pafi1', 'alb1', 'hema1', 'bili1', 'crea1', 'sod1', 'pot1',
       'paco21', 'ph1', 'swang1', 'wtkilo1', 'dnr1', 'ninsclas', 'resp',
       'card', 'neuro', 'gastr', 'renal', 'meta', 'hema', 'seps', 'trauma',
       'ortho', 'adld3p', 'urin1', 'race', 'income', 'ptid'],
      dtype='object')

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5735 entries, 0 to 5734
Data columns (total 63 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5735 non-null   int64  
 1   cat1        5735 non-null   object 
 2   cat2        1200 non-null   object 
 3   ca          5735 non-null   object 
 4   sadmdte     5735 non-null   int64  
 5   dschdte     5734 non-null   float64
 6   dthdte      3722 non-null   float64
 7   lstctdte    5735 non-null   int64  
 8   death       5735 non-null   object 
 9   cardiohx    5735 non-null   int64  
 10  chfhx       5735 non-null   int64  
 11  dementhx    5735 non-null   int64  
 12  psychhx     5735 non-null   int64  
 13  chrpulhx    5735 non-null   int64  
 14  renalhx     5735 non-null   int64  
 15  liverhx     5735 non-null   int64  
 16  gibledhx    5735 non-null   int64  
 17  malighx     5735 non-null   int64  
 18  immunhx     5735 non-null   int64  
 19  transhx     5735 non-null  

In [16]:
metadata = SingleTableMetadata()

In [17]:
metadata.detect_from_dataframe(data)
metadata

{
    "columns": {
        "Unnamed: 0": {
            "sdtype": "id"
        },
        "cat1": {
            "sdtype": "categorical"
        },
        "cat2": {
            "sdtype": "categorical"
        },
        "ca": {
            "sdtype": "categorical"
        },
        "sadmdte": {
            "sdtype": "numerical"
        },
        "dschdte": {
            "sdtype": "numerical"
        },
        "dthdte": {
            "sdtype": "numerical"
        },
        "lstctdte": {
            "sdtype": "numerical"
        },
        "death": {
            "sdtype": "categorical"
        },
        "cardiohx": {
            "sdtype": "categorical"
        },
        "chfhx": {
            "sdtype": "categorical"
        },
        "dementhx": {
            "sdtype": "categorical"
        },
        "psychhx": {
            "sdtype": "categorical"
        },
        "chrpulhx": {
            "sdtype": "categorical"
        },
        "renalhx": {
            "sdtype": "categorical

In [18]:
# metadata.save_to_json('metadata.json')

# in the future, you can reload the metadata object from the file
metadata = Metadata.load_from_json('metadata.json')

C:\Users\etudiant\Documents\Martin\tVAE_project\SDV\sdv\metadata\metadata.py:37: UserWarning: You are loading an older SingleTableMetadata object. This will be converted into the new Metadata object with a placeholder table name ('table'). Please save this new object for future usage.
  warnings.warn(


## Trainning + genration

In [ ]:
synthesizer = CTGANSynthesizer(metadata)
synthesizer.fit(data)

# custom_synthesizer = CTGANSynthesizer(
#    metadata,
#   epochs=1000,
#    verbose=True)
# custom_synthesizer.fit(real_data)

In [47]:
synthetic_data = synthesizer.sample(num_rows=500)
synthetic_data.to_csv('synthetic_data.csv', index=False)


In [48]:
synthetic_data = pd.read_csv('synthetic_data.csv')
synthetic_data.head(10)

,Unnamed: 0,cat1,cat2,ca,sadmdte,dschdte,dthdte,lstctdte,death,cardiohx,...,meta,hema,seps,trauma,ortho,adld3p,urin1,race,income,ptid
0,6329845,COPD,MOSF w/Sepsis,No,11724,12409.0,11441.0,11210,Yes,0,...,No,No,No,No,No,NaN,NaN,white,Under $11k,479
1,3039607,MOSF w/Sepsis,NaN,No,11526,12464.0,11727.0,12634,No,0,...,Yes,No,No,No,No,NaN,NaN,white,$25-$50k,1034
2,2669183,CHF,NaN,No,11523,12339.0,NaN,12232,No,0,...,No,No,No,No,No,1.0,2923.498989,black,Under $11k,213
3,7685588,ARF,NaN,Yes,11646,12236.0,NaN,12644,No,0,...,No,No,Yes,No,No,NaN,NaN,white,$11-$25k,856
4,15982309,ARF,MOSF w/Sepsis,No,11707,11610.0,11740.0,11947,Yes,0,...,No,No,Yes,No,No,NaN,NaN,white,Under $11k,925
5,13844016,Coma,MOSF w/Sepsis,No,10927,12477.0,11786.0,11237,Yes,0,...,No,No,No,No,No,0.0,NaN,white,$11-$25k,44264
6,9066854,CHF,MOSF w/Sepsis,No,11309,11019.0,11725.0,11806,Yes,0,...,No,No,Yes,No,No,NaN,NaN,white,$11-$25k,6
7,7366710,CHF,NaN,No,11174,10799.0,NaN,11837,Yes,0,...,No,No,No,No,No,NaN,2125.840809,white,$11-$25k,2674
8,15747194,ARF,NaN,No,11123,12400.0,11745.0,12354,No,0,...,No,No,Yes,No,No,NaN,NaN,white,Under $11k,6701
9,8448111,CHF,NaN,No,11363,11775.0,11458.0,11206,Yes,0,...,No,No,No,No,No,NaN,2122.285857,white,$11-$25k,9


## Evaluation

In [21]:
diagnostic = run_diagnostic(
    real_data=data,
    synthetic_data=synthetic_data,
    metadata=metadata
)

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 63/63 [00:00<00:00, 1341.83it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 130.22it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%



In [56]:
diagnostic.get_details('Data Validity')

,Column,Metric,Score
0,Unnamed: 0,KeyUniqueness,1.0
1,cat1,CategoryAdherence,1.0
2,cat2,CategoryAdherence,1.0
3,ca,CategoryAdherence,1.0
4,sadmdte,BoundaryAdherence,1.0
...,...,...,...
57,ortho,CategoryAdherence,1.0
58,adld3p,CategoryAdherence,1.0
59,urin1,BoundaryAdherence,1.0
60,race,CategoryAdherence,1.0


## Quality report

In [ ]:
diagnostic_report = run_diagnostic(
    real_data=data,
    synthetic_data=synthetic_data,
    metadata=metadata)

#diagnostic_report.get_details(property_name='Data Validity')


# check data validity, relationship validity, Structure

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 63/63 [00:00<00:00, 1403.72it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 250.62it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%



In [54]:
quality_report = evaluate_quality(
    data,
    synthetic_data,
    metadata
)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 63/63 [00:00<00:00, 283.41it/s]|
Column Shapes Score: 85.66%

(2/2) Evaluating Column Pair Trends: |██████████| 1953/1953 [00:07<00:00, 260.35it/s]|
Column Pair Trends Score: 81.27%

Overall Score (Average): 83.47%



In [ ]:
fig = get_column_plot(
    real_data=data,
    synthetic_data=synthetic_data,
    column_name='income',  # can make list # ['income', 'age']
    
)

fig.show()

# image DL in folder project

In [ ]:
# Dthdte = Date of Death

fig = get_column_plot(
    real_data=data,
    synthetic_data=synthetic_data,
    column_name='Dthdte',  # dont work ??
    
)

fig.show()

### 1. Column Shapes

In [57]:
quality_report.get_details('Column Shapes')

,Column,Metric,Score
0,cat1,TVComplement,0.811824
1,cat2,TVComplement,0.930269
2,ca,TVComplement,0.938443
3,sadmdte,KSComplement,0.783020
4,dschdte,KSComplement,0.802461
...,...,...,...
56,ortho,TVComplement,0.997221
57,adld3p,TVComplement,0.843126
58,urin1,KSComplement,0.685335
59,race,TVComplement,0.937582


In [ ]:
fig = get_column_plot(
    real_data=data,
    synthetic_data=synthetic_data,
    column_name='degree_perc',        # dont work ??
)    

fig.show()

#### KSComplement (numeric variable)

In [51]:
real_table = pd.read_csv('rhc.csv')
synthetic_table = pd.read_csv('synthetic_data.csv')

column_name = 'age'

ks_complement_score = KSComplement.compute(
    real_data=real_table[column_name],
    synthetic_data=synthetic_table[column_name]
)

print(f'KSComplement score for column {column_name}: {ks_complement_score}')


KSComplement score for column age: 0.7555483870967742


#### TVComplement (categorial variable)

In [53]:
column_name = 'income'

TVComplement.compute(
    real_data=real_table[column_name],
    synthetic_data=synthetic_table[column_name]
)

print(f'KSComplement score for column {column_name}: {ks_complement_score}')


KSComplement score for column income: 0.7555483870967742


### 2. Column Pair Trends

In [73]:
quality_report.get_visualization('Column Pair Trends')
quality_report

In [ ]:
fig = get_column_pair_plot(
    real_data=data,
    synthetic_data=synthetic_data,
    metadata=metadata,
    column_names=['XXXX', 'XXXX'], # choose columns
    )
    
fig.show()

#### Individual Metrics

In [ ]:
MissingValueSimilarity.compute(
    real_data=data['dthdte'],
    synthetic_data=synthetic_data['dthdte']   #dthdte = date of death
)

0.9749973844812554

In [ ]:
# tells us if the synthetic data contains any exact copies of the real data
NewRowSynthesis.compute(
    real_data=data,
    synthetic_data=synthetic_data,
    metadata=metadata.to_dict()            # metadata are not in the right format (missing a key value pair)
)